### 了解如何實作與使用N_gram模型
N_gram模型是基於統計的基礎語言模型，在這次的課程中，我們會學習到如何使用pyhon來實作N_gram的語言模型。
課程中的範例我們會使用[傲慢與偏見的英文文本](https://www.gutenberg.org/ebooks/1342)來進行。

---
### 進行資料清洗
在搭建語言模型前，需要將文本資料根據需求進行清洗，像是去除標點符號、將文字正規化(大小寫轉換)等等

In [28]:
#讀取文本資料

import urllib, requests

books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']


url_template = f'https://www.gutenberg.org/cache/epub/{book}/pg{book}.txt'
response = requests.get(url_template)
txt = response.text

#檢查文本
print(len(txt), ',', txt[:50] , '...')

717572 , ﻿The Project Gutenberg EBook of Pride and Prejudic ...


在開始搭建N-gram模型之前，先對文本進行清洗
(移除非英文與數字字元，且將所有的字轉為小寫)

In [29]:
import re

words = re.split('[^A-Za-z]+', txt.lower())
words = [x for x in words if x != ''] # 移除空字串

print(len(words))

125897


---
### 建立詞頻字典
N-gram模型所需的機率需要使用詞頻來做計算，因此先對文本進行詞頻計算。
這裡使用Unigram與Bigram進行

In [35]:
# unigram
unigram_frequecy = dict()

for word in words:
    unigram_frequecy[word] = unigram_frequecy.get(word, 0) + 1  
    # dict.get()取得特定 key 的 value，若無則回傳 0
    
# 根據詞頻排序, 並轉換為(word, count)格式
unigram_frequecy = sorted(unigram_frequecy.items(), key=lambda word_count: word_count[1], reverse=True)
# dict.items() = 轉為 iterable (dict -> tuple)
# lambda word_count: word_count[1] = 將每個 tuple 命名為 word_count，排列依據為其 index=1 項目

# 查看詞頻前10的字詞
unigram_frequecy[:10]

[('the', 4507),
 ('to', 4243),
 ('of', 3730),
 ('and', 3658),
 ('her', 2225),
 ('i', 2070),
 ('a', 2012),
 ('in', 1937),
 ('was', 1847),
 ('she', 1710)]

In [38]:
# bigram
bigram_frequency = dict()

for i in range(0, len(words)-1):
    bigram_frequency[tuple(words[i:i+2])] = bigram_frequency.get(tuple(words[i:i+2]), 0) + 1
    
# 根據詞頻排序, 並轉換為(word, count)格式
bigram_frequency = sorted(bigram_frequency.items(), key=lambda words_count: words_count[1], reverse=True)

# 查看詞頻前10的字詞
bigram_frequency[:10]

[(('of', 'the'), 491),
 (('to', 'be'), 445),
 (('in', 'the'), 397),
 (('i', 'am'), 303),
 (('mr', 'darcy'), 273),
 (('to', 'the'), 268),
 (('of', 'her'), 261),
 (('it', 'was'), 251),
 (('of', 'his'), 235),
 (('she', 'was'), 212)]

---
### 搭建N-gram模型
接下來可以隨機選取一個初始的單字，再利用建立好的N-gram模型(這裡我們採用bigram)，
來預測機率最高的10個字組成的句子，若沒有找到接續的詞，則中斷預測。

這裡我們建立的Bigram詞頻表為(word_pairs, count)格式，其中word_pairs為(word_1, word_2)。

In [39]:
def do_bigram_prediction(bigram_freq, start_word, num_words):
    pred_words = [start_word]
    word = start_word
    for i in range(num_words):
        # Find Next word
        word = next((word_pairs[1] for (word_pairs, count) in bigram_freq if word_pairs[0] == word), None)
        
        if not word:
            break
        else:
            pred_words.append(word)
    return pred_words

In [41]:
# 隨機選取起始字
import random

start_word = random.choice(words)
print(f'初始字: {start_word}')

pred_words = do_bigram_prediction(bigram_frequency, start_word, 10)
print(f"預測句子: {' '.join(pred_words)}")

初始字: to
預測句子: to be so much as to be so much as to


---
### 加入選取權重
依照上述的模型，會產生的問題是，如果開頭的字都相同(ex: of)，則後續的預測句子都一定會相同，
為了修正這樣的問題，我們可以利用N-gram詞頻中的頻率當作權重，增加隨機性，以確保後續預測的句子都不一定會相同。

In [42]:
# 權重選取
def weighted_choice(choices):
    total = sum(w for c, w in choices)
    r = random.uniform(0, total)
    upto = 0
    for c, w in choices:
        if upto + w > r:
            return c
        upto += w
    
def do_bigram_weighted_prediction(bigram_freq, start_word, num_words):
    pred_words = [start_word]
    word = start_word
    for i in range(num_words):
        # 選取所有符合條件的2gram
        words_candidates = [word_pairs_count for word_pairs_count in bigram_freq if word_pairs_count[0][0] == word]
        
        if not words_candidates:
            break
        else:
            #根據加權機率選取可能的字詞
            pred_words.append(weighted_choice(words_candidates)[1])
            
    return pred_words

In [43]:
start_word = 'of'
print(f'初始字: {start_word}')

pred_words = do_bigram_weighted_prediction(bigram_frequency, start_word, 10)
print(f"預測句子: {' '.join(pred_words)}")

初始字: of
預測句子: of concealing many them success elizabeth her shame some first the


In [44]:
start_word = 'of'
print(f'初始字: {start_word}')

pred_words = do_bigram_weighted_prediction(bigram_frequency, start_word, 10)
print(f"預測句子: {' '.join(pred_words)}")

初始字: of
預測句子: of your youth my his course the necessity you her saying


可以發現，相同的起始字，也可能得到不同的預測結果。

---
接下來我們可以根據上述的搭建方式，來建立泛化的N-gram模型(可自由選定N值)，根據N值的不同可以得到如Unigram(N=1)，Bigram(N=2)，Trigram(N=3)等。

In [45]:
def generateNgram(N):
    gram_frequency = dict()
    
    # 避免N值過大，導致記憶體崩潰問題(先設定N < 100)
    assert N > 0 and N < 100
    
    # 建立N-gram的頻率字典
    for i in range(len(words)-(N-1)):
        gram_frequency[tuple(words[i:i+N])] = gram_frequency.get(tuple(words[i:i+N]), 0) + 1

    # 根據詞頻排序, 並轉換為(word, count)格式
    gram_frequency = sorted(gram_frequency.items(), key=lambda words_count: words_count[1], reverse=True)
    
    return gram_frequency

In [46]:
# 建立Trigram
trigram_frequency = generateNgram(3)

# 查看詞頻前10的字詞
trigram_frequency[:10]

[(('i', 'do', 'not'), 62),
 (('i', 'am', 'sure'), 62),
 (('project', 'gutenberg', 'tm'), 57),
 (('as', 'soon', 'as'), 55),
 (('she', 'could', 'not'), 50),
 (('that', 'he', 'had'), 37),
 (('it', 'would', 'be'), 34),
 (('in', 'the', 'world'), 34),
 (('i', 'am', 'not'), 32),
 (('the', 'project', 'gutenberg'), 31)]

In [47]:
#建立N-gram預測function
def do_ngram_weighted_prediction(gram_freq, start_word, num_words):
    pred_words = [start_word]
    word = start_word
    for i in range(num_words):
        # 選取所有符合條件
        words_candidates = [word_pairs_count for word_pairs_count in gram_freq if word_pairs_count[0][0] == word]
        
        if not words_candidates:
            break
        else:
            #根據加權機率選取可能的字詞
            pred_words.append(weighted_choice(words_candidates)[1])
            
    return pred_words

In [48]:
start_word = 'of'
print(f'初始字: {start_word}')

pred_words = do_ngram_weighted_prediction(trigram_frequency, start_word, 10)
print(f"預測句子: {' '.join(pred_words)}")

初始字: of
預測句子: of her a finding the his course knowing a the his


---
### 使用NLTK套件搭建N-gram模型

NLTK (Natural Language Toolkit) 是一個 python 的自然語言處理套件，內含許多應用，這裡我們會使用 NLTK 來建立 N-gram 模型。

首先需要進行套件安裝，只需要執行

```bash
pip install nltk
```

In [52]:
import re
import collections
from nltk import ngrams

In [53]:
#讀取文本資料

import urllib, requests

books = {'Pride and Prejudice': '1342',
         'Huckleberry Fin': '76',
         'Sherlock Holmes': '1661'}

book = books['Pride and Prejudice']


url_template = f'https://www.gutenberg.org/cache/epub/{book}/pg{book}.txt'
response = requests.get(url_template)
txt = response.text

#檢查文本
print(len(txt), ',', txt[:50] , '...')

717572 , ﻿The Project Gutenberg EBook of Pride and Prejudic ...


In [54]:
words = re.split('[^A-Za-z]+', txt.lower())
words = [x for x in words if x != ''] # 移除空字串

print(len(words))

125897


In [57]:
# 建立 bigram 詞庫 (注意這裡回傳的是 generator)
# ((the, project), ('project', 'gutenberg')...)
bigram_frequency = ngrams(words, n=2) 

# 計算詞庫內的詞頻
bigram_frequency = collections.Counter(bigram_frequency)

In [63]:
bigram_frequency

Counter({('the', 'project'): 31,
         ('project', 'gutenberg'): 87,
         ('gutenberg', 'ebook'): 4,
         ('ebook', 'of'): 2,
         ('of', 'pride'): 7,
         ('pride', 'and'): 16,
         ('and', 'prejudice'): 6,
         ('prejudice', 'by'): 3,
         ('by', 'jane'): 11,
         ('jane', 'austen'): 4,
         ('austen', 'this'): 1,
         ('this', 'ebook'): 6,
         ('ebook', 'is'): 2,
         ('is', 'for'): 3,
         ('for', 'the'): 163,
         ('the', 'use'): 6,
         ('use', 'of'): 8,
         ('of', 'anyone'): 2,
         ('anyone', 'anywhere'): 2,
         ('anywhere', 'at'): 2,
         ('at', 'no'): 6,
         ('no', 'cost'): 2,
         ('cost', 'and'): 2,
         ('and', 'with'): 28,
         ('with', 'almost'): 2,
         ('almost', 'no'): 2,
         ('no', 'restrictions'): 2,
         ('restrictions', 'whatsoever'): 2,
         ('whatsoever', 'you'): 2,
         ('you', 'may'): 45,
         ('may', 'copy'): 2,
         ('copy', 'it'): 